使用一对多的多分类策略，为每个状态建立单独的分类器，然后分别训练

- 配置全局参数可以任意指定要训练的模型和对应的数据集
    - 保存生成的数据集并且要支持从使用之前生成的数据集
未完成部分
- 配置全局参数可以针对指定的模型进行保存和读取
    - 模型的保存和读取，要支持保存不同版本，同时留下模型的相关数据，方便后续选择出最优的模型
- 拆分训练集和验证集
- 配置整合函数main

In [1]:
from StatusChecker.TraditionalStatusChecker import TraditionalStatusChecker
import os
from datetime import datetime

current_train_tag = TraditionalStatusChecker.ASC_STATUS_FIGHTING
preset_positive_data_count_min = 100
learn_rate = 0.01


# 重新生成datetime.now()
log_time = datetime.now()
current_time = f"{log_time.year}-{log_time.month}-{log_time.day}" \
               f"-{log_time.hour}-{log_time.minute}-{log_time.second}"
# 日志保存位置
log_path = os.path.join(os.getcwd(), '..', 'log')
# 训练集日志保存位置
train_set_list_log_base_path = os.path.join(log_path, 'dataset_list', current_time)
os.makedirs(train_set_list_log_base_path, exist_ok=True)

# 基础数据集路径
train_set_base_path = os.path.join(os.getcwd(), '..', 'dataset', 'processed')
# 验证集路径
validation_data_base_path = os.path.join(os.getcwd(), '..', 'dataset', 'validation')
# 数据集列表保存路径
data_set_list_base_path = train_set_list_log_base_path
os.makedirs(data_set_list_base_path, exist_ok=True)

# 模型检查点保存路径
module_checkpoint_base_path = os.path.join(log_path, 'model', current_time)
os.makedirs(module_checkpoint_base_path, exist_ok=True)
# 模型训练数据保存路径
module_train_data_base_path = os.path.join(log_path, 'train_log', current_time)
os.makedirs(module_train_data_base_path, exist_ok=True)


print(f"setup finished, timestap is {current_time}")

setup finished, timestap is 2020-8-3-1-27-19


In [2]:
status_list = list(filter(lambda name: (name.startswith("ASC_STATUS_") and name != 'ASC_STATUS_UNKNOWN'),
                                  dir(TraditionalStatusChecker)))
status_list = list(map(lambda name: getattr(TraditionalStatusChecker, name), status_list))

print(status_list)
print(f"total {len(status_list)} status")
# tag_dic = status_list
# tag_dic = ["level_selection",
#            "restore_sanity_medicine",
#            "restore_sanity_stone",
#            "team_up",
#            "fighting",
#            "battle_settlement",
#            "annihilation_settlement",
#            "level_up"]


['annihilation_settlement', 'battle_settlement', 'fighting', 'level_selection', 'level_up', 'restore_sanity_medicine', 'restore_sanity_stone', 'team_up']
total 8 status


根据训练目标选取数据集
- 正向数据集选取目标标签，数量选取所有的或者指定阈值
- 反向数据集选取其余的标签数据，数量取正向数据集长度平均至每个TAG，不够的暂时全上

In [3]:
import os
import random

#TODO 如果说数据量不平衡的话，最后那个数据会影响很多东西把
def generate_train_data_list():
    # 生成数据数量
    data_count = {current_train_tag: min(
        len(os.listdir(os.path.join(train_set_base_path, current_train_tag))),
        preset_positive_data_count_min
    )}
    print(f"needed positive data count: {data_count[current_train_tag]}")
    negative_data_count_need_each = data_count[current_train_tag] / (len(status_list) - 1)
    print(f"needed negative data count: {negative_data_count_need_each}")
    for status in status_list:
        if status == current_train_tag:
            continue
        # 统计，取可用数据数量和当前所需目标数量的最小值
        data_count[status] = min(
            negative_data_count_need_each,
            len(os.listdir(os.path.join(train_set_base_path, status)))
        )
    print(f"decided data count use: {data_count}")
    # 根据数量开始处理数据路径
    # 一个是要方便之后拿到TAG，一个是考虑要不要每次随机选取数据，不要每次都是开头的那些数据
    # TAG可以随时从文件名拿到，随机选取数据用random生成一个不重复的数组，然后将它们作为实际的数据索引
    # 跟上完整log以配合输出
    # 正向数据生成
    train_set_positive_data_set = random.choices(
        os.listdir(
            os.path.join(train_set_base_path, current_train_tag)
        ), k=data_count[current_train_tag]
    )
    # 负面数据生成
    train_set_negative_data_set = []
    for status in status_list:
        if status == current_train_tag:
            continue
        train_set_negative_data_set += random.choices(
                os.listdir(
                    os.path.join(train_set_base_path, status)
                ), k=int(data_count[status])
            )
    # 输出本次数据集到log文件夹

    print(f"start log at {current_time}, status: {current_train_tag}")
    with open(os.path.join(train_set_list_log_base_path, f"train-data-set-list-{current_train_tag}.log"), "w") as log_file:
        log_file.write("\n===========positive start===============\n")
        log_file.writelines(train_set_positive_data_set)
        log_file.write("\n===========positive end=================\n")
        log_file.write("\n===========negative start===============\n")
        log_file.writelines(train_set_negative_data_set)
        log_file.write("\n===========negative end=================\n")
    print("log finished")
    # 获得最终的数据列表
    data_set = train_set_positive_data_set + train_set_negative_data_set
    # 保存生成的数据列表

    train_set_list_filename = f"train-set-file-list-{current_train_tag}.pt"
    torch.save(data_set, os.path.join(data_set_list_base_path, train_set_list_filename))
    print(f"saved train set list in train-set-file-list.pt")
    return data_set

# 从本地读取已生成的数据列表
def load_data_list(filename: str):
    return torch.load(os.path.join(data_set_list_base_path, filename))

# 用于生成数据对应的TAG
def get_image_tag(filename:str) -> int:
    # for index, tag in enumerate(status_list):
    #     if tag in filename:
    #         return index
    for tag in status_list:
        if tag in filename:
            if tag == current_train_tag:
                return 1
            else:
                return 0

def get_image_tag_origin(filename:str) -> str:
    for tag in status_list:
        if tag in filename:
            return tag

def generate_target_list(train_file_list:list):

    # 取得对应的标签数据
    return list(map(get_image_tag, train_file_list))

配置 DataSet

In [4]:
import cv2 as cv
from torch.utils.data import Dataset

def default_loader(image_name:str):
    image_loaded = cv.imread(
        os.path.join(train_set_base_path, get_image_tag_origin(image_name), image_name),
        cv.IMREAD_GRAYSCALE)
    # print(f"load image {image_name}")
    # print(image_tensor.size())
    return image_loaded

def validation_path_loader(image_name:str):
    image_loaded = cv.imread(
        os.path.join(validation_data_base_path, image_name),
        cv.IMREAD_GRAYSCALE)
    # print(f"load image {image_name}")
    # print(image_tensor.size())
    return image_loaded

class CustomDataSet(Dataset):
    def __init__(self, image_name_list:list, targets:list, loader=default_loader)-> None:
        self.images = image_name_list
        self.targets = targets
        self.loader = loader
    def __getitem__(self, index: int):
        image = self.loader(self.images[index])
        image = torch.tensor(image, device=torch.device('cuda'), dtype=torch.float32).unsqueeze(0)
        target = self.targets[index]
        target = torch.tensor(target, dtype=torch.float32, device=torch.device('cuda'))
        return image, target
    def __len__(self)-> int:
        return len(self.images)

def get_train_set(train_file_list: list) -> Dataset:
    target_list = generate_target_list(train_file_list)

    print(len(train_file_list))
    print(train_file_list[0])
    print(len(target_list))
    print(target_list[0])

    train_set = CustomDataSet(train_file_list, target_list)
    print(train_set)
    print(train_set.__getitem__(3))
    print(train_set.__len__())
    return train_set


def get_validation_set()->Dataset:
    validation_file_list = os.listdir(validation_data_base_path)
    target_list = list(map(get_image_tag, validation_file_list))
    validation_set = CustomDataSet(validation_file_list, target_list, loader=validation_path_loader)
    return validation_set

构建 DataLoader

In [5]:
from torch.utils.data import DataLoader
def get_train_loader(train_set:Dataset) -> DataLoader:
    train_loader = DataLoader(train_set, batch_size=1, shuffle=True)
    print(train_loader)
    print(iter(train_loader).next()[0].size())
    print(iter(train_loader).next()[1].size())
    return train_loader

def get_validation_loader(validation_set:Dataset) -> DataLoader:
    validation_loader = DataLoader(validation_set, batch_size=1, shuffle=True)
    print(validation_loader)
    print(iter(validation_loader).next()[0].size())
    print(iter(validation_loader).next()[1].size())
    return validation_loader

定义模型

In [6]:
import torch

class Lambda(torch.nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func
    def forward(self, xb):
        return self.func(xb)

def get_module():
    module = torch.nn.Sequential(
        torch.nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=3),
        torch.nn.ReLU(),
        torch.nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=3),
        torch.nn.ReLU(),
        torch.nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=3),
        torch.nn.ReLU(),
        torch.nn.AdaptiveAvgPool2d([16, 36]),
        Lambda(lambda xb: xb.view(-1, 10*16*36)),
        torch.nn.Linear(10*16*36, 16*36),
        torch.nn.Linear(16*36, 36),
        torch.nn.Linear(36, 1),
        torch.nn.Softmax(),
    ).to(torch.device('cuda'))
    optimize = torch.optim.SGD(module.parameters(), lr=learn_rate)
    return module, optimize

def get_loss_function():
    loss_func = torch.nn.functional.binary_cross_entropy
    return loss_func

fit() 函数以及模型的保存和恢复

In [7]:
import numpy


def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    # return loss.item(), len(xb)
    return loss, 1
def fit(epochs:int, module:torch.nn.Module,
        loss_function, optimize, train_loader, validate_loader):
    index = 0
    with open(os.path.join(module_train_data_base_path, 'train.log'), 'w') as log:
        for epoch in range(epochs):
            module.train()
            for xb, yb in train_loader:
                index = index + 1
                if index%20 == 0:
                    print(f"start index {index} data")
                train_loss, _ = loss_batch(module, loss_function, xb, yb, optimize)
                if index%20 == 0:
                    log.write(f"epoch: {epoch}, index: {index}, train_loss: {train_loss}\n")
            module.eval()
            with torch.no_grad():
                losses, nums = zip(
                    *[loss_batch(module, loss_function, xb, yb)
                      for xb, yb in validate_loader]
                )
            validation_loss = numpy.sum(numpy.multiply(losses, nums)) / numpy.sum(nums)
            print(epoch, validation_loss)
            log.write(f"epoch: {epoch}, index: {index}, val_loss: {validation_loss}\n")


准备启动

In [8]:
# def main():
print(status_list)


train_file_list = generate_train_data_list()
# train_file_list = load_data_list('')
train_set = get_train_set(train_file_list)
validation_set = get_validation_set()
train_loader = get_train_loader(train_set)
validation_loader = get_validation_loader(validation_set)
module, optimize  = get_module()
loss_function = get_loss_function()

fit(epochs=2, module=module, loss_function=loss_function,
    optimize=optimize, train_loader=train_loader, validate_loader=validation_loader)


['annihilation_settlement', 'battle_settlement', 'fighting', 'level_selection', 'level_up', 'restore_sanity_medicine', 'restore_sanity_stone', 'team_up']
needed positive data count: 100
needed negative data count: 14.285714285714286
decided data count use: {'fighting': 100, 'annihilation_settlement': 14.285714285714286, 'battle_settlement': 14.285714285714286, 'level_selection': 14.285714285714286, 'level_up': 7, 'restore_sanity_medicine': 14.285714285714286, 'restore_sanity_stone': 14.285714285714286, 'team_up': 14.285714285714286}
start log at 2020-8-3-1-27-19, status: fighting
log finished
saved train set list in train-set-file-list.pt
191
fighting-6554.png
191
1
(tensor([[[ 42.,  40.,  41.,  ...,  40.,  40.,  41.],
         [ 39.,  38.,  39.,  ...,  39.,  38.,  39.],
         [ 41.,  39.,  40.,  ...,  40.,  39.,  41.],
         ...,
         [ 39.,  38.,  39.,  ..., 103., 100.,  97.],
         [ 38.,  38.,  38.,  ..., 103., 100.,  97.],
         [ 37.,  36.,  37.,  ...,  97.,  93.,

D:\software\Anaconda3\envs\auto_fight\lib\site-packages\torch\nn\modules\container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
<ipython-input-7-468beef88a3a>:5: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])) is deprecated. Please ensure they have the same size.
  loss = loss_func(model(xb), yb)
